# Derive the Dynamics and Sensor Model

## Derive Dynamics Model

Import all packages used to derive the equations of motion of the system

In [ ]:
import sympy as sym
import numpy as np
sym.init_printing()

Define a function that converts a vector in coordinate frame A to a coordinate frame B where the orientation of A in B is described by a set of angles: roll, pitch, yaw.

In [ ]:
def change_of_coords(v_inA, roll, pitch, yaw):
    # Get the rotation matrices from the roll, pitch, and yaw values
    cx = np.cos(roll)
    sx = np.sin(roll)
    cy = np.cos(pitch)
    sy = np.sin(pitch)
    cz = np.cos(yaw)
    sz = np.sin(yaw)
    Rx = np.array([[1., 0., 0.],
                   [0., cx, -sx],
                   [0., sx, cx]])
    Ry = np.array([[cy, 0., sy],
                   [0., 1., 0.],
                   [-sy, 0., cy]])
    Rz = np.array([[cz, -sz, 0.],
                   [sz, cz, 0.],
                   [0., 0., 1.]])

    # Combine the individual rotations into a single transform
    R_ofA_inB = Rz@Ry@Rx

    # Apply the transform the the original vector in frame A
    v_inB = R_ofA_inB @ v_inA

    # Return the same vector in frame B
    return v_inB

Define all the system constants (system parameters)

In [ ]:
# Mass and moment of inertia of the bus
mb = 6.
Ixxb = 10.
Iyyb = 10.
Izzb = 16.

# Radius of the bus
rb = 2.2

# Mass and moment of inertia of each wheel
mw = 1.
Ixxw = 0.075
Iyyw = 0.075
Izzw = 0.125

Generate the symbols that define the state of the spacecraft. Also, create the rotation matrices that define the orientation of the spacecraft in the world coordinate system.

In [ ]:
# Define roll (phi), pitch (theta), and yaw (psi) angles
phi, theta, psi = sym.symbols('phi, theta, psi')

# Define angular velocities
wx_inB, wy_inB, wz_inB = sym.symbols('omega_x, omega_y, omega_z')

# Define angular velocity vector written in body coordinates
w_inB = sym.Matrix([[wx_inB],
                    [wy_inB],
                    [wz_inB]])

# Define roll, pitch, and yaw rotation matrices that describe the bus's orientation in the 
# world frame
Rx = sym.Matrix([[1,            0,             0],
                 [0, sym.cos(phi), -sym.sin(phi)],
                 [0, sym.sin(phi),  sym.cos(phi)]])
Ry = sym.Matrix([[ sym.cos(theta), 0, sym.sin(theta)],
                 [              0, 1,              0],
                 [-sym.sin(theta), 0, sym.cos(theta)]])
Rz = sym.Matrix([[sym.cos(psi), -sym.sin(psi), 0],
                 [sym.sin(psi),  sym.cos(psi), 0],
                 [           0,             0, 1]])

Generate the symbols that define the input to the system. Also sum the together to get the net input vector.

In [ ]:
# Define torques
tau_1, tau_2, tau_3, tau_4 = sym.symbols('tau_1, tau_2, tau_3, tau_4')

# Multiply torque scalars by their directions (in each wheel's frame)
# to get the torque applied to each wheel as a vector
t1_inT1 = tau_1 * sym.Matrix([[0.],
                              [0.],
                              [1.]])
t2_inT2 = tau_2 * sym.Matrix([[0.],
                              [0.],
                              [1.]])
t3_inT3 = tau_3 * sym.Matrix([[0.],
                              [0.],
                              [1.]])
t4_inT4 = tau_4 * sym.Matrix([[0.],
                              [0.],
                              [1.]])

# Change the torque on the wheel vectors from their own body coordinates to bus coordinates
t1_inB = change_of_coords(t1_inT1, 0.,       0., 1.570796)
t2_inB = change_of_coords(t2_inT2, 2.094395, 0., 2.617994)
t3_inB = change_of_coords(t3_inT3, 2.094395, 0., 4.712389)
t4_inB = change_of_coords(t4_inT4, 2.094395, 0., 0.523599)

# Sum the torque vectors to get the total torque on the spacecraft by the wheels 
# in bus coordinates.
# We use minus because the torque on the bus is in the opposite direction as the torque
# applied to the wheels (Newton's third law)
tau_inB = -t1_inB - t2_inB - t3_inB - t4_inB
tau_inB = sym.simplify(tau_inB)
print("Torques applied to spacecraft in spacecraft-fixed coordinates:")
sym.N(tau_inB, 3)

Apply the parallel axis theorem to convert the moment's of inertia about each wheel's center to the center of the bus. We assume that each wheel has no moment of inertia and can be modeled as a point mass. This is valid because the moment of inertia about each wheel is small compared to the inertia imposed by their location

In [ ]:
# First we get the position of the center of mass of each wheel in bus coordinates
p1_inB = np.round(np.array(rb*(t1_inB/tau_1)).astype(np.float64).flatten(),6)
p2_inB = np.round(np.array(rb*(t2_inB/tau_2)).astype(np.float64).flatten(),6)
p3_inB = np.round(np.array(rb*(t3_inB/tau_3)).astype(np.float64).flatten(),6)
p4_inB = np.round(np.array(rb*(t4_inB/tau_4)).astype(np.float64).flatten(),6)
ps_inB = [p1_inB, p2_inB, p3_inB, p4_inB]

# Get the total moment of inertia
Ixx = Ixxb
Iyy = Iyyb
Izz = Izzb
for p_inB in ps_inB:
    # Get the distance from the x axis and the resultant point inertia
    dx = np.linalg.norm(p_inB[1:])
    ixx = mw * dx**2
    Ixx = Ixx + ixx
    
    # Get the distance from the y axis and the resultant point inertia
    dy = np.linalg.norm(p_inB[0:3:2])
    iyy = mw * dy**2
    Iyy = Iyy + iyy
    
    # Get the distance from the z axis and the resultant point inertia
    dz = np.linalg.norm(p_inB[0:2])
    izz = mw * dz**2
    Izz = Izz + izz

# Assemble the moment of inertia matrix
I_inB = sym.Matrix([[Ixx, 0.,  0.],
                    [0.,  Iyy, 0.],
                    [0.,  0.,  Izz]])
print("Moment of inertia of spacecraft written in principal axes:")
sym.N(I_inB, 3)

Use [Euler's Equations](https://en.wikipedia.org/wiki/Euler%27s_equations_(rigid_body_dynamics)) to generate the equations of motion that relate angular velocity to input torque.

In [ ]:
# Apply Euler's equation to get the time derivative of the angular velocities of the
# spacecraft in the spacecraft frame
w_inB_dot = I_inB.inv() @ (tau_inB - w_inB.cross(I_inB@w_inB))
w_inB_dot = sym.simplify(w_inB_dot)
print("Time derivative of the angular velocities in the spacecraft-fixed frame:")
sym.N(w_inB_dot, 3)

Calculate the roll, pitch, and yaw rates as a function of the orientation and body-fixed angular rates.

In [ ]:
# Define the transformation that takes roll, pitch, and yaw rates to body-fixed angular rates
ex = sym.Matrix([[1], [0], [0]])
ey = sym.Matrix([[0], [1], [0]])
ez = sym.Matrix([[0], [0], [1]])
M = sym.Matrix.hstack(ex, Rx.T@ey, (Ry@Rx).T@ez)

# Invert the transformation to get a matrix that takes body-fixed angular rates to 
# roll, pitch, and yaw rates
M_inv = sym.simplify(M.inv())

# Convert the body-fixed angular rates to roll, pitch, and yaw rates
rpy_dot = sym.simplify(M_inv@w_inB)
print("Roll, pitch, and yaw rate as a function of orientation and body-fixed angular rates:")
sym.N(rpy_dot,3)

Assemble all differential equations into a single system of equations.

In [ ]:
# Assemble the system of equations
f = sym.simplify(sym.Matrix.vstack(rpy_dot, w_inB_dot))

The equations of motion have this form:

$$\begin{bmatrix} \dot{\phi} \\ \dot{\theta} \\ \dot{\psi} \\ \dot{\omega_x} \\ \dot{\omega_y} \\ \dot{\omega_z} \end{bmatrix}=f\left(\phi, \theta, \psi, \omega_x, \omega_y, \omega_z, \tau_1, \tau_2, \tau_3, \tau_4\right)$$

Here is the function $f$:

In [ ]:
sym.N(f,3)

## Derive Sensor Model

Here we will develope the system of equations that relates the sensed states to the states used in the system dynamics. We start by defining where each star is in space. We can do this using its right ascension, $\alpha$, and the declination, $\delta$ in the sky. This type of coordinate system is called an [equitorial celestial coordinate system](https://en.wikipedia.org/wiki/Equatorial_coordinate_system).

In [ ]:
# Create a variable for the right ascension and declination of the ith star
alpha_i, delta_i = sym.symbols('alpha_i, delta_i')

We can now use these coordinates to calculate the $i$th star's location in the scope image. We do this using a pinhol camera model. To start, we transform the $i$th star's position from world coordinates to spacecraft-fixed body coordinates. This is done using the spacecraft's orientation and the $i$th star's position in world coordinates.

In [ ]:
# Calculate the DEPTH NORMALIZED cartesian coordinates of the ith star
# in the fixed world coorinates using the equitorial celestial coordinates
p_ofi_inW = sym.Matrix([[sym.cos(alpha_i)*sym.cos(delta_i)],
                        [sym.sin(alpha_i)*sym.cos(delta_i)],
                        [sym.sin(delta_i)]])
print("Depth-normalized cartesian coordinates of the ith star in world coordinates:")
p_ofi_inW

In [ ]:
# Calculate the orientation of the spacecraft using it's roll, pitch, and yaw angles
R_ofS_inW = sym.simplify(Rz @ Ry @ Rx)
print("Orientation of spacecraft in world:")
R_ofS_inW

In [ ]:
# Invert the orientation of the spacecraft to get a transform that takes vectors
# in world coordinates to vectors in spacecraft-fixed coordinates.
# As it turns out, the inverse of a valid rotation matrix, R, is its transpose
R_ofW_inS = R_ofS_inW.T

# Get the position of the ith star in spacecraft-fixed coordinates
p_ofi_inS = sym.simplify(R_ofW_inS @ p_ofi_inW)
print("Depth-normalized cartesian coordinates of the ith star in spacecraft-fixed coordinates:")
p_ofi_inS

Now that we have the depth-normalized coordinates of the $i$th star in spacecraft-fixed coordinates, we can estimate where the star will appear in the starscope's image.

In [ ]:
# Define the radius of the star scope
rc = 0.8/2.1

# Calculate the star scope coordinates, q,  of the ith star
# This step is why we can use depth normalized coordinates
# We divide the y,z coords by the x coord. This causes any
# constants, such as depth, to drop out.
g = sym.simplify( (1/rc) * sym.Matrix([[p_ofi_inS[1] / p_ofi_inS[0]],
                                       [p_ofi_inS[2] / p_ofi_inS[0]]]) )

This gives us our sensor model for each star. The model has the form:
$$
q_i = g(\phi, \theta, \psi, \alpha_{i}, \delta_{i}),
$$
where $q_i$ is the star scope coordinates of the $i$th star. Here is the function $g$:

In [ ]:
g